# Compare the forecasts with real records

In [61]:
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

# Load the environment variables from the .env file
load_dotenv('.env')

# Get the values of host, user, pswd, db, and schema from the environment variables
host = os.getenv('host')
user = os.getenv('user')
pswd = os.getenv('pswd')
db = os.getenv('db')
schema = os.getenv('schema')


# Use the values as needed
engine = create_engine(
    f"postgresql://{user}:{pswd}@{host}/{db}?options=-csearch_path%3D{schema}", echo=False)
conn = engine.connect()

In [62]:
import datetime as dt
import pytz

now = dt.datetime.now(pytz.timezone('Asia/Singapore'))
date = now.strftime("%Y-%m-%d")
time = now.strftime("%H:%M")

period = int(now.strftime("%H")) * 2 + int(now.strftime("%M")) // 30 + 1

# date = '2024-03-25' # A hard-coded value for testing
# period = 33 # A hard-coded value for testing

# print(f"Now is {date} {time} Period {period}")

In [63]:
predicted = pd.read_sql(f"""
SELECT "Date", "Period", "Predicted_Net_Demand"
FROM public."Predicted_Net_Demand"
""", conn)
predicted.sort_values(by=['Date', 'Period'], inplace=True)
# predicted.tail(5)

In [64]:
earliest_date = predicted.iloc[0, 0].strftime("%Y-%m-%d")
rt_dpr = pd.read_sql(f"""
SELECT "Date", "Period", "Demand", "TCL", "Transmission_Loss"
FROM public."Real_Time_DPR"
WHERE ("Date" >= '{earliest_date}')
ORDER BY "Date", "Period"  
""", conn)
rt_dpr.sort_values(by=['Date', 'Period'], inplace=True)
rt_dpr.reset_index(drop=True, inplace=True)
# rt_dpr.head(5)

In [65]:
vc_per = pd.read_sql('SELECT * FROM public."VCData_Period"', conn)
# vc_per.iloc[[0, -1]]

In [66]:
import holidays

# Calculate required data fields

sg_holidays = holidays.country_holidays('SG')

rt_dpr['Total Demand'] = rt_dpr['Demand'] + rt_dpr['TCL'] + rt_dpr['Transmission_Loss']
view = rt_dpr[['Date', 'Period', 'Total Demand']].copy()

def find_tcq(row):
    # print(row)
    date_obj = dt.datetime.strptime(str(row['Date']), '%Y-%m-%d')
    year = date_obj.year
    quarter = (date_obj.month - 1) // 3 + 1
    
    period = row['Period']
    
    isWeekend = 1 if date_obj.isoweekday() > 5 else 0
    isPublicHoliday = date_obj in sg_holidays
    
    if isWeekend or isPublicHoliday:
        # print(f"Date: {date_obj} isWeekend: {isWeekend} isPublicHoliday: {isPublicHoliday}")
        tcq = vc_per[(vc_per['Year'] == year) & (vc_per['Quarter'] == quarter) & (vc_per['Period'] == period)]['TCQ_Weekday'].values[0] / 1000
    else:
        tcq = vc_per[(vc_per['Year'] == year) & (vc_per['Quarter'] == quarter) & (vc_per['Period'] == period)]['TCQ_Weekend_PH'].values[0] / 1000

    # print(f"Date: {date_obj} TCQ: {tcq}")
    return tcq

view['TCQ'] = view.apply(lambda row: find_tcq(row), axis=1)
view['Net Demand'] = view['Total Demand'] - view['TCQ']
view.reset_index(drop=True, inplace=True)
# view.head(2)

In [67]:
def return_real(row):
    date = row['Date']
    
    period = row['Period']
    real = view[(view['Date'] == date) & (view['Period'] == period)]
    real = real["Net Demand"].values[0]
    # print(real)

    return real

predicted["Real"] = predicted.apply(lambda row: return_real(row), axis=1)
predicted["Error (R-P)"] = predicted["Real"] - predicted["Predicted_Net_Demand"]

In [68]:
print(f"Now is {date} {time} Period {period}")
predicted

Now is 2024-03-27 14:26 Period 29


,Date,Period,Predicted_Net_Demand,Real,Error (R-P)
0,2024-03-27,30,6583.65625,6510.33361,-73.32264


In [69]:
conn.close()